In [1]:
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from math import modf,log

In [2]:
# connection to the database and ingesting the bounds count table

conn = pyodbc.connect(r'DRIVER={SQL Server Native Client 11.0};' +
 ('SERVER=dbr42xlarge.cofwcrzxs5v2.us-east-2.rds.amazonaws.com,1433;'   +
  'DATABASE=Dashboard;'      +
  'UID=xxxxxxxxx;'           +
  'PWD=xxxxxxxxx').format(
                server= 'dbr42xlarge.cofwcrzxs5v2.us-east-2.rds.amazonaws.com',
                  port= 1433,
              database= 'Dashboard',
              username= 'xxxxxxxxxx',
              password= 'xxxxxxxxxx')
)
bound = pd.read_sql_query('select * from dbo.bound_Counts', conn)

conn.close()

In [3]:
# connection to the database and ingesting the quotes count table

conn = pyodbc.connect(r'DRIVER={SQL Server Native Client 11.0};' +
 ('SERVER=dbr42xlarge.cofwcrzxs5v2.us-east-2.rds.amazonaws.com,1433;'   +
  'DATABASE=Dashboard;'      +
  'UID=xxxxxxxxx;'           +
  'PWD=xxxxxxxxx').format(
                server= 'dbr42xlarge.cofwcrzxs5v2.us-east-2.rds.amazonaws.com',
                  port= 1433,
              database= 'Dashboard',
              username= 'xxxxxxxxx',
              password= 'xxxxxxxxx')
)
quote = pd.read_sql_query('select * from dbo.quotes_count', conn)

conn.close()

In [4]:
# connection to the database and ingesting the claims table

conn = pyodbc.connect(r'DRIVER={SQL Server Native Client 11.0};' +
 ('SERVER=dbr42xlarge.cofwcrzxs5v2.us-east-2.rds.amazonaws.com,1433;'   +
  'DATABASE=Dashboard;'      +
  'UID=xxxxxxxxx;'           +
  'PWD=xxxxxxxxx').format(
                server= 'dbr42xlarge.cofwcrzxs5v2.us-east-2.rds.amazonaws.com',
                  port= 1433,
              database= 'Dashboard',
              username= 'xxxxxxxxx',
              password= 'xxxxxxxxx')
)
claims = pd.read_sql_query('select * from dbo.claims', conn)

conn.close()

In [5]:
# connection to the database and ingesting the claims table

conn = pyodbc.connect(r'DRIVER={SQL Server Native Client 11.0};' +
 ('SERVER=dbr42xlarge.cofwcrzxs5v2.us-east-2.rds.amazonaws.com,1433;'   +
  'DATABASE=Dashboard_RS;'      +
  'UID=xxxxxxxxx;'           +
  'PWD=xxxxxxxxx').format(
                server= 'dbr42xlarge.cofwcrzxs5v2.us-east-2.rds.amazonaws.com',
                  port= 1433,
              database= 'Dashboard_RS',
              username= 'xxxxxxxxx',
              password= 'xxxxxxxxx')
)
claim_latlong = pd.read_sql_query('select * from dbo.claims_latlong', conn)

conn.close()

In [6]:

# create a date column and drop the PropertyCountyCode column 

def prepare_table(df):
    
    c = list(df.columns)
    df1 = pd.DataFrame(df)
    
    if "BindDt" in c:
        
        df1 = df1.drop(columns=['PropertyCountyCd'],axis=1)
        df1["Date"] = pd.to_datetime(df1['BindDt'])
        df1.rename(columns={'Bound':'Count','TotalPremium':'Premium'},inplace=True)
        df1['Identify'] = 'B'
        
    elif "QuoteDt" in c:
        
        df1 = df1.drop(columns=['PropertyCountyCode'],axis=1)
        df1["Date"] = pd.to_datetime(df1['QuoteDt'])
        df1.rename(columns={'Quotes_Count':'Count'},inplace=True)
        df1['Identify'] = 'Q'
        
    else:
        df1["Date"] = pd.to_datetime(df1['reporteddt'])
        df1.rename(columns={'Incurred':'Premium'},inplace=True)
        df1['Identify'] = 'C'
        
        df2 = df1.copy()
        df2['Peril_Group'] = 'ALL'
        df1 = pd.concat([df1,df2])
        
    return df1
    

In [7]:
quote_table = prepare_table(quote)

In [8]:
bound_table = prepare_table(bound)

In [9]:
claims_table = prepare_table(claims)

In [10]:
claim_latlong["Date"] = pd.to_datetime(claim_latlong['reporteddt'])
claim_latlong.rename(columns={'Incurred':'Premium'},inplace=True)
claim_latlong['Identify'] = 'C'

In [11]:
# create functions to extract the Daily, weekly and monthly data from the main table

class graph_count:
    def __init__(self,df,col,value,n):
        self.df = df
        self.col = col
        self.value = value
        self.time = n
        

 # creating the range of dates as selected

    def date_list_daily(self):
        
        end = datetime.today() - timedelta(days=1)
        
        start = end - timedelta(days=self.time)

        date_list = pd.date_range(start.date(), end.date(), freq='D')
        date_list = date_list.strftime("%Y-%m-%d")
        rng = date_list.values.tolist()
        date_range = [datetime.strptime(d,'%Y-%m-%d') for d in rng]
        
        return sorted(date_range)
    
    
 # creating the range of weeks as selected

    def date_list_weekly(self):
        
        end = datetime.today()

        start = end - timedelta(weeks=self.time)

        date_list = pd.date_range(start.date(), end.date(), freq='W')
        date_list = date_list.strftime("%Y-%m-%d")
        rng = date_list.values.tolist()
        date_range = [datetime.strptime(d,'%Y-%m-%d') for d in rng]
        
        return sorted(date_range)
    
    
    
 # filtering the data on the basis of selected dates

    def filter_df_daily(self):
        
        df_range = self.df[self.df['Date'] >= min(self.date_list_daily())]
        
        return df_range
    
 
 # filtering the data on the basis of selected weeks

    def filter_df_weekly(self):
        df_range = self.df[self.df['Date'] > min(self.date_list_weekly())]
        
        return df_range
    
    
 # filtering the data on the basis of selected months   
    
    def filter_df_monthly(self):
        self.df['Current_Date'] = datetime.today().date()
        self.df['L_Month'] = self.df['Date'].map(lambda x: x.month)
        self.df['L_Year'] = self.df['Date'].map(lambda x: x.year)
        self.df['C_Month'] = self.df['Current_Date'].map(lambda x: x.month)
        self.df['C_Year'] = self.df['Current_Date'].map(lambda x: x.year)
        self.df['Month_Diff'] = 12*(self.df['C_Year']-self.df['L_Year']) + (self.df['C_Month'] - self.df['L_Month'])
        
        df = self.df.drop(['Current_Date','L_Month','C_Month','L_Year','C_Year'],axis=1)
        
        df_range = df[df['Month_Diff'] <= self.time]
        
        return df_range

    
 # filtering the data on the basis of selected quarters      
    
    def filter_df_quarter(self):
        
        self.df['Current_Date'] = datetime.today().date()
        self.df['L_Month'] = self.df['Date'].map(lambda x: x.month )
        self.df['L_Year'] = self.df['Date'].map(lambda x: x.year)
        self.df['C_Month'] = self.df['Current_Date'].map(lambda x: x.month)
        self.df['C_Year'] = self.df['Current_Date'].map(lambda x: x.year)
        self.df['Month_Diff'] = 12*(self.df['C_Year']-self.df['L_Year']) + (self.df['C_Month'] - self.df['L_Month'])
        
        df = self.df.drop(['Current_Date','L_Month','C_Month','L_Year','C_Year'],axis=1)
        
       
        if modf(datetime.today().date().month/3)[0] == 0:
            n = self.time*3
        elif modf(datetime.today().date().month/3)[0] > 0.5 and modf(datetime.today().date().month/3)[0] < 0.8:
            n = self.time*3 - 1 
        else:
            n = self.time*3 - 2
        
        
        df_range = df[df['Month_Diff'] < n ]
        
        df_range['Quarter'] = df_range['Date'].map(lambda x: int(x.year*100 + (int(x.month/3)+1)) if x.month%3 != 0 else int(x.year*100 + x.month/3) )
        
        return df_range
    
    
 # creating a table with required columns for daily
        
    def daily(self):
        
        daily = self.filter_df_daily()
        
        df_daily = daily.filter([self.col,self.value,'Date'],axis=1)
        daily_pivot = pd.pivot_table(df_daily,values = self.value,
                          index = 'Date',columns= self.col ,aggfunc=np.sum)
        daily_pivot = daily_pivot.fillna(value=0)
        daily_pivot_df = pd.DataFrame(daily_pivot)
        daily_pivot_df.index = [datetime.date(d) for d in daily_pivot_df.index]
        
        return daily_pivot_df
    
    
  # creating a table with required columns for weekly   
    
    def weekly(self):
        
        date_range = self.date_list_weekly()
       
        weekly = self.filter_df_weekly()
        
        weekly['Week'] = 0
        
        for index,row in weekly.iterrows():
            for i in np.arange(0,len(date_range),1):
                
                if i < len(date_range)-1:
                   
            
                    if row['Date'] > date_range[i] and row['Date'] <= date_range[i+1]:
                        weekly.at[index,'Week'] = date_range[i+1].date()
                        
                elif i == len(date_range)-1:
                    
                    if row['Date'] > date_range[i]:
                        
                        weekly.at[index,'Week'] = datetime.today().date()
                        
        
                        
        df_weekly = weekly.filter([self.col,self.value,'Week'],axis=1)
        weekly_pivot = pd.pivot_table(df_weekly,values = self.value,
                          index = 'Week',columns= self.col ,aggfunc=np.sum)
        weekly_pivot = weekly_pivot.fillna(value=0)
        weekly_pivot_df = pd.DataFrame(weekly_pivot)                
        
        
        return weekly_pivot_df
    

 # creating a table with required columns for monthly
    
    def monthly(self):
        
        monthly = self.filter_df_monthly()
        monthly['YearMonth'] = monthly['Date'].map(lambda x: 100*x.year + x.month)
        df_monthly = monthly.filter([self.col,self.value,'YearMonth'],axis=1)
        monthly_pivot = pd.pivot_table(df_monthly,values = self.value,
                          index = 'YearMonth',columns= self.col ,aggfunc=np.sum)
        monthly_pivot = monthly_pivot.fillna(value=0)
        monthly_pivot_df = pd.DataFrame(monthly_pivot)
        
        return monthly_pivot_df 
    
    
    def quarterly(self):
        
        quarterly = self.filter_df_quarter()
        
        df_quarterly = quarterly.filter([self.col,self.value,'Quarter'],axis=1)
        quarterly_pivot = pd.pivot_table(df_quarterly,values = self.value,
                          index = 'Quarter',columns= self.col ,aggfunc=np.sum)
        quarterly_pivot = quarterly_pivot.fillna(value=0)
        quarterly_pivot_df = pd.DataFrame(quarterly_pivot)
        
        return quarterly_pivot_df
    
    
    

## Dashboard

In [12]:
import dash 
import dash_html_components as html
import dash_core_components as dcc
import plotly.graph_objs as go
from dash.dependencies import Input,Output
#import dash_table_experiments as dt
import dash_table as dt

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__,external_stylesheets=external_stylesheets)



mapbox_access_token = 'xxxxxxxxx'


app.css.append_css({'external_url': 'https://cdn.rawgit.com/plotly/dash-app-stylesheets/2d266c578d2a6e8850ebce48fdb52759b2aef506/stylesheet-oil-and-gas.css'})

def transform_value(value):
    return 10**value


loss_description = [{'label':'Fire','value':'Fire'},
                    {'label':'Flood','value':'Flood'},
                    {'label':'Freezing Water','value':'Freezing water'},
                    {'label':'Hail','value':'Hail'},
                    {'label':'Hurricane','value':'Hurricane'},
                    {'label':'Lightining','value':'Lightining'},
                    {'label':'Tornado','value':'Tornado'},
                    {'label':'Water Damage','value':'Water Damage'},
                    {'label':'Wind','value':'Wind'},
                    {'label':'Other','value':'Other'}          
                                     
                   ]



app.layout = html.Div([html.Div([
                                html.Div(
                                [
                                    html.H1(
                                    'Maison Dashboard',
                                    className = 'eight columns',
                                    )
                                ])
                                ,
                                html.Div(
                                [
                                    dcc.DatePickerSingle(
                                                        id='date-picker-single',
                                                        date=datetime.today().date()
                                                        )
                                   
                                ], className = 'one columns',
                                    style = {'height': '100',
                                            'width': '225',
                                            'float': 'right',
                                                'position': 'relative',})   
    
                        ],className='row'),
                        html.Div([
                                    html.Div(
                                    [
                                        html.P('Select the sheet:'),
                                        dcc.Dropdown(
                                                    id = 'selection',
                                                    options = [
                                                        
                                                        {'label':'Quotes','value':'Quotes'},
                                                        {'label':'Bound','value':'Bound'},
                                                        {'label':'Claims','value':'Claims'}
                                                    ],
                                                    value='Quotes'

                                        ),
                                        dcc.RadioItems(
                                                    id = 'period',
                                                    options = [
                                                        {'label':'Daily','value':'Days'},
                                                        {'label':'Weekly','value':'Weeks'},
                                                        {'label':'Monthly','value':'Months'},
                                                        {'label':'Quarterly','value':'Quarters'}
                                                    ],
                                                    value='Days',
                                                    labelStyle = {'display':'inline-block'}
                                        ),
                                        dcc.Slider(
                                                    id='range',
                                                    min=0,
                                                    max=12,
                                                    value=7,
                                                    marks={i: i for i in range(0,13,1) }
                                        )

                                    ],className='six columns'
                                    ),
                                    html.Div(
                                    [
                                        html.P('Select the Category:'),
                                        dcc.Dropdown(
                                                    id='category',
                                                    options = [
                                                        {'label':'State','value':'State'},
                                                        {'label':'Super Region Name','value':'Super_Region_Name'},
                                                        {'label':'Product Code','value':'ProductCd'}

                                                    ],
                                                    value='State'
                                        ),
                                        html.P('Select the cause of LOSS: '),
                                        dcc.Dropdown(
                                                    id='COL',
                                                    options = [
                                                        {'label':'All','value':'ALL'},
                                                        {'label':'Non-Weather Water','value':'Non-Weather Water'},
                                                        {'label':'Fire','value':'Fire'},
                                                        {'label':'AOP','value':'AOP'},
                                                        {'label':'Weather','value':'Weather'}
                                                    ],
                                                    multi = True,
                                                    value='ALL'
                                        )
                                        
                                    ], className='six columns'
                                    )
                            
                            
                            
                        ],className = 'row'),
                        html.Div([
                                    html.Div(
                                    [
                                        dcc.Graph(id='main_graph',
                                                 config = {
                                                     'displayModeBar':False  # hiding the floating toolbar
                                                 }),
                                        
                                    ],
                                        className='six columns',
                                        style={'margin-top':10}
                                    ),
                                    html.Div(
                                    [
                                        dcc.Graph(id='cause_of_loss'),
                                             
                                    ],
                                        className='six columns',
                                        style={'margin-top':10}
                                    
                                    )
                            
                        ],className = 'row'),
                        html.Div([
                                    html.Div(
                                    [
                                        html.P('Counts'),
                                        dt.DataTable(id='count-data',
                                                    n_fixed_columns=1,
                                                    sorting=True,
                                                    style_cell={'textAlign': 'center'}, 
                                                    content_style='grow',
                                                    style_as_list_view=True
                                                    )
                                    ],
                                        className='four columns',
                                        style= {'marginRight':10,'margin-top':10,'display':'inline-block','width':'25%'}
                                    
                                    ),
                                    html.Div(
                                    [
                                        html.P('Premium/Incurred Loss'),
                                        dt.DataTable(id='premium-data',
                                                    
                                                    n_fixed_columns=1,
                                                    sorting=True,
                                                    style_cell={'textAlign': 'center'},
                                                     content_style='grow',
                                                     style_as_list_view=True
                                                    )
                                    ],
                                        className='four columns',
                                        style= {'marginLeft':30,'marginRight':10,'margin-top':10,
                                                'display':'inline-block','width':'25%'}
                                    
                                    ),
                                    html.Div(
                                    [
                                        dcc.Graph(id='Incurred_Loss',
                                                  config = {
                                                      'displayModeBar':False
                                                  })                            
                                    ], className = 'four columns',
                                        style = {'margin-top':10,'width':'45%','margin-left':10}
                                    
                                    
                                    )
                            ], className = 'row'),
                            
                            html.Div(                                       
                                        [
                                            html.P('Loss Description'),
                                            dcc.Dropdown(id='Description',
                                                        options = loss_description,
                                                        multi = True,
                                                        placeholder='Select a cause of loss'),

                                        ],
                                            className='row',
                                            style={'margin-top':5,'width':'25%'}

                                        ),
                            html.Div(
                                        [
                                            html.P('Loss Amount'),
                                            dcc.RangeSlider(id='range_select',
                                                            
                                                            max = 6.1761,
                                                            step = 0.01,
                                                           marks={i: '{}'.format(10**i) for i in range(0,7,1)},
                                                            value = [0,6]
                                                            
                                                           ),
                                            html.Div(id='range_value',style={'margin-top':20,'margin-bottom':20})
                                        ],
                                            
                                            className = 'row',
                                            style={'marginTop':5,'width':'100%'}
                                            
                                        ),
                                      
                            html.Div(
                                        [
                                        dcc.Graph(id='loss-graph',
                                        config = {'scrollZoom':True},
                                        figure={ 'data': [], 'layout': []} 
                                                 )
                                        ],
                                            className='row',
                                            style={'margin-top':5,'display':'inline-block','width':'100%'}            

                                ),
                            ], className = 'row')
                       # ] , className='row')
    
#])

# callback for main graph that is bound - quote ratio table

@app.callback(Output('main_graph','figure'),
             [Input('category','value'),
              Input('range','value'),
              Input('period','value')])
def bq_chart(cat,n,period):
    if period == 'Weeks':
        b = graph_count(bound_table,cat,'Count',n).weekly()
        q = graph_count(quote_table,cat,'Count',n).weekly()
        
    elif period == 'Months':
        b = graph_count(bound_table,cat,'Count',n).monthly()
        q = graph_count(quote_table,cat,'Count',n).monthly()
        
    elif period == 'Quarters':
        b = graph_count(bound_table,cat,'Count',n).quarterly()
        q = graph_count(quote_table,cat,'Count',n).quarterly()
        
    else:
        b = graph_count(bound_table,cat,'Count',n).daily()
        q = graph_count(quote_table,cat,'Count',n).daily()

    bq = (b/q)*100
    
    bq.fillna(value=0,inplace=True)
    xa = 'Period of {} {}'.format(n,period)
    
    traces = []
    
    for i in bq.columns:
        traces.append(
                        go.Scatter(
                                    x = bq.index,
                                    y = bq[i],
                                    name = i ,
                                    mode = 'lines',
                                    marker = dict(colorscale='Jet')
                                )
                    )
    return {
        'data':traces,
        'layout': go.Layout(title = 'Bound-Quote Ratio',
                            margin = {
                                        'l':35,
                                        'r':35,
                                        't':45,
                                        'b':45
                                    },
                            autosize=True,
                            xaxis = {'title':xa, 
                                      'type':'category',
                                      'tickvals':bq.index,
                                      'ticktext':bq.index},
                             yaxis = {'title':'Percentage (%)'}
                        )
    }


# callback for claims with cause of loss

@app.callback(Output('cause_of_loss','figure'),
             [Input('COL','value'),
             Input('range','value'),
             Input('period','value')])
def callback_claims_col(col1,rng,periodt):
    if periodt == 'Weeks':
        tbl = graph_count(claims_table,'Peril_Group','Count',rng).weekly()
        
    elif periodt == 'Months':
        tbl = graph_count(claims_table,'Peril_Group','Count',rng).monthly()
        
    elif periodt == 'Quarters':
        tbl = graph_count(claims_table,'Peril_Group','Count',rng).quarterly()
        
    else:
        tbl = graph_count(claims_table,'Peril_Group','Count',rng).daily()
        
        
        
    xa = 'Period of {} {}'.format(rng,periodt)
    
    traces1 = []
    col1 = list(col1)
    
    try:    
    
    
        for i in col1:
            traces1.append(
                            go.Bar(
                                    x = tbl.index,
                                    y = tbl[i],
                                    name = i                         
                                  )

                            )

        return{
            'data':traces1,
            'layout':go.Layout(title = 'Claims By Cause Of LOSS',
                               barmode = 'group',
                                margin = {
                                        'l':50,
                                        'r':35,
                                        't':45,
                                        'b':45
                                    },
                               xaxis = {'title':xa,
                                        'type':'category',
                                        'tickvals':tbl.index,
                                        'ticktext':tbl.index},

                               yaxis = {'title': 'Count of Claims'}

                               )        
            }
    except:
        return{
            'data':[go.Bar(
                            x = tbl.index,
                            y = tbl['ALL'],
                            name = i
            )],
            'layout':go.Layout(title = 'Claims By Cause Of LOSS',
                               barmode = 'group',
                                margin = {
                                        'l':50,
                                        'r':35,
                                        't':45,
                                        'b':45
                                    },
                               autosize=True,
                               xaxis = {'title':xa,
                                        'type':'category',
                                        'tickvals':tbl.index,
                                        'ticktext':tbl.index},

                               yaxis = {'title': 'Count of Claims'}

                               )
        }

    
@app.callback(Output('Incurred_Loss','figure'),
             [Input('COL','value'),
             Input('range','value'),
             Input('period','value')])
def callback_claims_incurred(col1,rng,periodt):
    if periodt == 'Weeks':
        tbl = graph_count(claims_table,'Peril_Group','Premium',rng).weekly()
        
    elif periodt == 'Months':
        tbl = graph_count(claims_table,'Peril_Group','Premium',rng).monthly()
        
    elif periodt == 'Quarters':
        tbl = graph_count(claims_table,'Peril_Group','Premium',rng).quarterly()
        
    else:
        tbl = graph_count(claims_table,'Peril_Group','Premium',rng).daily()
        
        
        
    xa = 'Period of {} {}'.format(rng,periodt)
    
    traces1 = []
    col1 = list(col1)
    
    try:    
    
    
        for i in col1:
            traces1.append(
                            go.Scatter(
                                    x = tbl.index,
                                    y = tbl[i],

                                    mode = 'lines',
                                    
                                    name = i                         
                                  )

                            )

        return{
            'data':traces1,
            'layout':go.Layout(title = 'Incurred Loss By Cause Of LOSS',
                               
                                margin = {
                                        'l':50,
                                        'r':35,
                                        't':45,
                                        'b':45
                                    },
                               xaxis = {'title':xa,
                                        'type':'category',
                                        'tickvals':tbl.index,
                                        'ticktext':tbl.index},

                               yaxis = {'title': 'Incurred_Loss',
                                       'hoverformat':',.0f'}

                               )        
            }
    except:
        return{
            'data':[go.Scatter(
                            x = tbl.index,
                            y = tbl['ALL'],
 
                            mode = 'lines',
                            name = i
            )],
            'layout':go.Layout(title = 'Incurred Loss By Cause Of LOSS',
                               barmode = 'group',
                                margin = {
                                        'l':50,
                                        'r':35,
                                        't':45,
                                        'b':45
                                    },
                               autosize=True,
                               xaxis = {'title':xa,
                                        'type':'category',
                                        'tickvals':tbl.index,
                                        'ticktext':tbl.index},

                               yaxis = {'title': 'Incurred_Loss',
                                       'hoverformat':',.0f'}

                               )
        }



    
# callback for count table


@app.callback([Output('count-data','data'),
               Output('count-data','columns')],
             [Input('selection','value'),
              Input('category','value'),
              Input('range','value'),
              Input('period','value')])
def callback_table(selection,category,rng,period):
    if selection == 'Bound':
        if period == 'Weeks':
            df = graph_count(bound_table,category,'Count',rng).weekly()
        elif period == 'Days':
            df = graph_count(bound_table,category,'Count',rng).daily()
        elif period == 'Months':
            df = graph_count(bound_table,category,'Count',rng).monthly()
        elif period == 'Quarters':
            df = graph_count(bound_table,category,'Count',rng).quarterly()    
    
    elif selection == 'Claims':
        if period == 'Weeks':
            df = graph_count(claims_table,category,'Count',rng).weekly()/2
        elif period == 'Days':
            df = graph_count(claims_table,category,'Count',rng).daily()/2
        elif period == 'Months':
            df = graph_count(claims_table,category,'Count',rng).monthly()/2
        elif period == 'Quarters':
            df = graph_count(claims_table,category,'Count',rng).quarterly()/2    
    else:
        if period == 'Weeks':
            df = graph_count(quote_table,category,'Count',rng).weekly()
        elif period == 'Days':
            df = graph_count(quote_table,category,'Count',rng).daily()
        elif period == 'Months':
            df = graph_count(quote_table,category,'Count',rng).monthly()
        elif period == 'Quarters':
            df = graph_count(quote_table,category,'Count',rng).quarterly()
            
            
    for i in df.columns:
        df[i]=df[i].map("{:,.0f}".format)
        
    df.reset_index(inplace=True)
    df.rename(columns={'index':period},inplace=True)
    column = [{"name":i,"id":i} for i in df.columns]
    data = df.to_dict("rows")
    return data,column

# callback for premium table
      
@app.callback([Output('premium-data','data'),
               Output('premium-data','columns')],
             [Input('selection','value'),
              Input('category','value'),
              Input('range','value'),
              Input('period','value')])
def callback_table(selection,category,rng,period):
    if selection == 'Bound':
        if period == 'Weeks':
            df = graph_count(bound_table,category,'Premium',rng).weekly()
        elif period == 'Days':
            df = graph_count(bound_table,category,'Premium',rng).daily()
        elif period == 'Months':
            df = graph_count(bound_table,category,'Premium',rng).monthly()
        elif period == 'Quarters':
            df = graph_count(bound_table,category,'Premium',rng).quarterly()    
            
    elif selection == 'Claims':
        if period == 'Weeks':
            df = graph_count(claims_table,category,'Premium',rng).weekly()/2
        elif period == 'Days':
            df = graph_count(claims_table,category,'Premium',rng).daily()/2
        elif period == 'Months':
            df = graph_count(claims_table,category,'Premium',rng).monthly()/2
        elif period == 'Quarters':
            df = graph_count(claims_table,category,'Premium',rng).quarterly()/2    
            
            
    else:
        if period == 'Weeks':
            df = graph_count(quote_table,category,'Premium',rng).weekly()
        elif period == 'Days':
            df = graph_count(quote_table,category,'Premium',rng).daily()
        elif period == 'Months':
            df = graph_count(quote_table,category,'Premium',rng).monthly()
        elif period == 'Quarters':
            df = graph_count(quote_table,category,'Premium',rng).quarterly()    
            
            
    for i in df.columns:
        df[i]=df[i].map("{:,.0f}".format)
    
    df.reset_index(inplace=True)
    df.rename(columns={'index':period},inplace=True)
    column=[{"name":i,"id":i} for i in df.columns]
    data = df.to_dict('rows')
    return data,column

# callback value for range slider

@app.callback(Output('range_value','children'),
              [Input('range_select','value')])   
def update_output(value):
    transformed_value = [transform_value(v) for v in value]
    return 'Min Value: {:,.0f}, Max Value: {:,.0f}'.format(
        transformed_value[0],
        transformed_value[1]
    )




#callback for claim map



@app.callback(Output('loss-graph','figure'),
             [Input('period','value'),
              Input('range','value'),
              Input('Description','value'),
             Input('range_select','value')])
def callback_map(period,rng,desc,val):
    if period == 'Weeks':
        df = graph_count(claim_latlong,'State','Count',rng).filter_df_weekly()
    elif period == 'Months':
        df = graph_count(claim_latlong,'State','Count',rng).filter_df_monthly()
    elif period == 'Quarters':
            df = graph_count(claim_latlong,'State','Count',rng).filter_df_quarter()
        
    else:
        df = graph_count(claim_latlong,'State','Count',rng).filter_df_daily()
        
    transformed_value = [transform_value(v) for v in val]


    df_f = df[['Date','Count','LossDescription','State','latitude','longitude','ProductCd','CountyName','Premium']]
    
    df_filter = df[df['Premium'].between(transformed_value[0],transformed_value[1],inclusive=True)]
    
   
   
    description_list = list(desc)
        
    df_map = df_filter[df_filter.LossDescription.isin(description_list)]
   
        
    
    map_data = df_map.groupby(['latitude','longitude','ProductCd','CountyName','State','LossDescription'],as_index=False).sum()
    
    cities = []
    

    
    for i in range(0,len(description_list),1):
        df_sub = map_data[map_data['LossDescription'] == description_list[i]]
        


        
        city = go.Scattermapbox(

            lon = df_sub['longitude'],
            lat = df_sub['latitude'],
            text = df_sub['ProductCd']+" : "+df_sub['CountyName'],
            hoverinfo = 'text',  # to just view the text excluding the longitude and latitude
            marker = go.scattermapbox.Marker(
                size = df_sub['Count']*3,
                opacity=0.7,
                colorscale='Jet',
     
            ),
            name = description_list[i])
        
        cities.append(city)
        
        
    layout = go.Layout(
                        title = go.layout.Title(
                            text = 'Claims in last {} {}'.format(rng,period)
                        ),                        
                        autosize=True,
                        margin = {
                            'l' : 0,
                            'b' : 0,
                            'r' : 0,
                            't' : 50
                            
                        },
                        showlegend = True,
                        mapbox = go.layout.Mapbox(
                                    accesstoken = mapbox_access_token,
                                    bearing = 0,
                                    center=go.layout.mapbox.Center(
                                    lat=31,
                                    lon=-99),
                            pitch=0,
                            zoom=4.2,
                            style='light'
                        
                        ),
                       
                    )    
        
    
    fig = go.Figure(data=cities, layout=layout)   
    return fig



if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/May/2019 10:41:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2019 10:41:20] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2019 10:41:20] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2019 10:41:20] "GET /_favicon.ico HTTP/1.1" 200 -
[2019-05-24 10:41:20,529] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\rsarkar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\rsarkar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\rsarkar\AppData\Local\Continuum\anaconda3\lib\site-packages\flask\app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\rsarkar\AppData\Local\Continuum